## Code for Data Generation

In [1]:
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import math

### Some Initialisations

In [2]:
#Material Parameters
E0 = 1
E1 = 1
eta = 100
total_time = 100

#Can be set as parameter (change it to 101)
time_array = np.linspace(0, total_time, 101)

delta_time = time_array[1]-time_array[0]
print(delta_time)

#Number of control points (can be changed to 2 or 3 for start)
num_cp = 4

#Position of control points (time)
cp_array = np.linspace(0, total_time, num = num_cp+2)

#Strain range
min_strain = -0.1
max_strain = 0.1

#Can be changed, increase or decrease
number_possible_strains = 11
strain_data_points = np.linspace(min_strain, max_strain, number_possible_strains)

#Number of combinations
num_combination = len(strain_data_points)**num_cp
print(num_combination)
print(strain_data_points)

1.0
14641
[-0.1  -0.08 -0.06 -0.04 -0.02  0.    0.02  0.04  0.06  0.08  0.1 ]


## Loading Matrix

In [11]:
# Function which returns subset or r length from n
import itertools
from itertools import combinations

holder = []

for p in itertools.product(range(0,len(strain_data_points)), repeat = num_cp):
    for j in range(num_cp):
        holder.append(strain_data_points[int(p[j])])

split_list = []

for index in range(0, len(holder), num_cp):
    split_list.append(holder[index:index+num_cp])
    
inner_array = np.asarray(split_list)

df = pd.DataFrame(inner_array)
df.insert(0, num_cp, 0)
df.insert(num_cp+1, num_cp+1, 0)
df.columns = ["start", "eps_1", "eps_2", "eps_3", "eps_4", "finish"]
df.head()

,start,eps_1,eps_2,eps_3,eps_4,finish
0,0,-0.1,-0.1,-0.1,-0.10,0
1,0,-0.1,-0.1,-0.1,-0.08,0
2,0,-0.1,-0.1,-0.1,-0.06,0
3,0,-0.1,-0.1,-0.1,-0.04,0
4,0,-0.1,-0.1,-0.1,-0.02,0


In [9]:
df.describe()

,start,eps_1,eps_2,eps_3,eps_4,finish
count,14641.0,1.464100e+04,1.464100e+04,1.464100e+04,1.464100e+04,14641.0
mean,0.0,-1.164745e-17,-5.823723e-18,-2.578211e-18,-2.523234e-18,0.0
std,0.0,6.324771e-02,6.324771e-02,6.324771e-02,6.324771e-02,0.0
min,0.0,-1.000000e-01,-1.000000e-01,-1.000000e-01,-1.000000e-01,0.0
25%,0.0,-6.000000e-02,-6.000000e-02,-6.000000e-02,-6.000000e-02,0.0
50%,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,0.0,6.000000e-02,6.000000e-02,6.000000e-02,6.000000e-02,0.0
max,0.0,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,0.0


In [12]:
column_selector = ["start", "eps_1", "eps_2", "eps_3", "eps_4", "finish"]
slope = np.zeros(num_cp+1)

number_loading_histories = number_possible_strains**num_cp

eps, eps_V, str, str_d, str_s, Wsto, Wdis, Wtotal, Esto, Edis, Etotal = (np.zeros((number_loading_histories, len(time_array)+1)) for _ in range(11))

for index, row in df.iterrows():
    for i in range(0, num_cp+1):
        slope[i] = (row[column_selector[i+1]]-row[column_selector[i]])/(total_time)*(num_cp+1)
    #print(index)
    
    #Loading, strain control
    for time in range(len(time_array)):
        for i in range(len(cp_array)-1):
            if time_array[time] < cp_array[i+1]:
                eps[index, time+1] = eps[index, time] + slope[i]*delta_time
                #print("Time = ", time_array[time+1], "Eps = ", eps[index, time+1])
                break
    
    #Pack into a function to provide more flexibility
    for n in range(len(time_array)):
        eps_V[index, n+1] = (eta*delta_time)/(eta + (E0 + E1)*delta_time)*(E0*eps[index, n+1]/eta + eps_V[index, n]/delta_time)
        str_s[index, n+1] = E1*eps_V[index, n+1]
        str_d[index, n+1] = eta*(eps_V[index, n+1] - eps_V[index, n])/delta_time
        str[index, n+1] = str_d[index, n+1] + str_s[index, n+1]
        Wsto[index, n+1] = 1/delta_time*(str_s[index, n+1]*(eps_V[index, n+1]-eps_V[index, n])+str[index, n+1]/E0*(str[index, n+1]-str[index, n]))
        Wdis[index, n+1] = 1/eta*str_d[index, n+1]**2
        Wtotal[index, n+1] = Wsto[index, n+1] + Wdis[index, n+1]
        Esto[index, n+1] = Esto[index, n]+Wsto[index, n+1]*delta_time
        Edis[index, n+1] = Edis[index, n]+Wdis[index, n+1]*delta_time
        Etotal[index, n+1] = Etotal[index, n]+Wtotal[index, n+1]*delta_time

In [13]:
%%time
##### Converting to DataFrame from Numpy, faster, please use this

column_names = ["load_index", "time", "eps", "eps_V", "str_s", "str_d", "str", "Wsto", "Wdis", "Wtotal", "Esto", "Edis", "Etotal"]
s = (101, 13)
arr_all = np.empty(s)

history = list()
for index, row in df.iterrows():
    arr_all = np.empty(s)
    arr_all[:, 0] = int(index)
    #print(index)
    l = np.transpose(time_array)
    arr_all[:, 1] = l
    item = [0,0, eps[index,:-1], eps_V[index,:-1], str_s[index,:-1], str_d[index,:-1], str[index,:-1], Wsto[index,:-1], Wdis[index,:-1], Wtotal[index,:-1], Esto[index,:-1], Edis[index,:-1], Etotal[index,:-1]]
    for i in range(2, 13):
        l = np.transpose(item[i])
        arr_all[:, i] = l
    
    history.append(pd.DataFrame(arr_all))
    
df_history = pd.concat(history)
df_history.columns = column_names

Wall time: 2.79 s


In [29]:
df_history.to_csv("benchmark(-0.2-0.1).dat", index=False)

In [14]:
df_history.describe()

,load_index,time,eps,eps_V,str_s,str_d,str,Wsto,Wdis,Wtotal,Esto,Edis,Etotal
count,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06,1.478741e+06
mean,7.320000e+03,5.000000e+01,-3.632619e-18,-5.958264e-19,-5.958264e-19,9.610104e-21,-7.688083e-19,8.917435e-06,1.434738e-05,2.326481e-05,1.269421e-03,7.162002e-04,1.985621e-03
std,4.226494e+03,2.915477e+01,4.598753e-02,1.004450e-02,1.004450e-02,3.787794e-02,4.091337e-02,1.761727e-04,1.866309e-05,1.811527e-04,1.165238e-03,6.754233e-04,1.607426e-03
min,0.000000e+00,0.000000e+00,-1.000000e-01,-3.800907e-02,-3.800907e-02,-1.385962e-01,-1.192981e-01,-1.151440e-03,0.000000e+00,-1.021016e-03,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.660000e+03,2.500000e+01,-3.200000e-02,-6.066841e-03,-6.066841e-03,-2.587362e-02,-2.796586e-02,-5.561728e-05,1.304892e-06,-4.910263e-05,4.112531e-04,1.268238e-04,6.741213e-04
50%,7.320000e+03,5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.733974e-06,6.694443e-06,2.912600e-06,9.304200e-04,5.706295e-04,1.701756e-03
75%,1.098000e+04,7.500000e+01,3.200000e-02,6.066841e-03,6.066841e-03,2.587362e-02,2.796586e-02,7.415300e-05,2.060569e-05,8.523299e-05,1.815298e-03,1.098381e-03,2.925859e-03
max,1.464000e+04,1.000000e+02,1.000000e-01,3.800907e-02,3.800907e-02,1.385962e-01,1.192981e-01,1.000892e-03,1.920891e-04,1.192981e-03,8.415575e-03,4.869809e-03,1.149955e-02
